# Neural Network Testing Framework
This notebook is intended to be a general framework for testing Neural Neworks on the EEG data set. While the initial iteration of this notebook is used for a simple Convolutional Neural Network (CNN), other networks using PyTorch can be implemented and assigned to the neural network variable. Many of the beginning cells are used to intialize for a Google Drive Colaboratory notebook GPU usability and can be ignored as necessary.

---



##Initialization
Google Drive access, PyTorch, etc.

In [0]:
!kill -9 -1

This section provides access to the user's Google drive.

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


This section creates a drive that links to the user's Google drive.

In [9]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


This section installs PyTorch.

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision

##Imports

Imports can be added as necessary.

In [0]:
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.cuda
from torch.utils.data import Dataset
from torch.autograd import Variable
from scipy import stats

##Classes

###EEGDataset
This class inherits the torch.utils.data.Dataset class to be used with the torch.utils.data.Dataloader class.

In [0]:
class EEGDataset(Dataset):
  """EEG dataset."""
  
  def __init__(self, x, y, transform=None):
    """
    Args:
      x (numpy array): Input data of shape 
                       num_trials x num_electrodes x num_time_bins.
      y (numpy array): Output data of shape num_trials x 1.
      transform (callable, optional): Optional transform to be applied.
    """
    self.x = x
    self.y = y
    self.transform = transform
    
  def __len__(self):
    return len(self.x)
  
  def __getitem__(self, idx):
    x_sample = torch.from_numpy(self.x[idx])
    y_sample = torch.IntTensor([int(self.y[idx])])
    
    if self.transform:
      pass #FIXME
    
    return x_sample, y_sample

###EEGMinimalContainer
This class holds a train and test EEGDataset. It processes the data into a (N, C, H, W) format in time.

In [0]:
class EEGMinimalContainer():
  """EEG container for training and testing datasets."""
  
  def __init__(self, data_dir, train_subject, seed=42):
    """
    Args:
      data_dir (string): Path to all A0iT_slice.mat files for i in [1, 9].
      train_subject(int): Subject to train on. If None, train on all.
      test_subject(int): Subject to test on. If None, test on all except for
                         train_subject. Only used if train_subject is not None.
    """
    self.X_train = None
    self.y_train = None
    self.X_test = None
    self.y_test = None
    self.train_dataset = None
    self.test_dataset = None
    np.random.seed(seed)
    
    # Step 1: Obtain Training Set
    A0iT = h5py.File(data_dir + ('/A0%dT_slice.mat' % (train_subject)), 'r')
    X = np.copy(A0iT['image'])
    X = np.expand_dims(X, axis=1)
    X = X[:, :, 0:22, :] 
    y = np.copy(A0iT['type'])
    y = y[0,0:X.shape[0]:1]
    y = np.asarray(y, dtype=np.int32)
    y -= 769
    remove_list = []
    for i in range(len(X)):
      if np.isnan(X[i]).any():
        remove_list.append(i)
    X = np.delete(X, remove_list, axis=0)
    y = np.delete(y, remove_list, axis=0)
    self.X_train = X
    self.y_train = y
    self.train_dataset = EEGDataset(X, y)
    # Step 2: Obtain Test Set
    self.X_test = {}
    self.y_test = {}
    self.test_dataset = {}
    for i in np.arange(1, 10):
      if i == train_subject:
        continue
      A0iT = h5py.File(data_dir + ('/A0%dT_slice.mat' % (i)), 'r')
      X = np.copy(A0iT['image'])
      X = np.expand_dims(X, axis=1)
      X = X[:, :, 0:22, :] 
      y = np.copy(A0iT['type'])
      y = y[0,0:X.shape[0]:1]
      y = np.asarray(y, dtype=np.int32)
      y -= 769
      remove_list = []
      for j in range(len(X)):
        if np.isnan(X[j]).any():
          remove_list.append(j)
      X = np.delete(X, remove_list, axis=0)
      y = np.delete(y, remove_list, axis=0)
      self.X_test[str(i)] = X
      self.y_test[str(i)] = y
      self.test_dataset[str(i)] = EEGDataset(X, y)
      
    print('EEGContainer X_train: ' + str(self.X_train.shape))
    print('EEGContainer y_train: ' + str(self.y_train.shape))
    for i in range(1, 10):
      if i == train_subject:
        continue
      print(('EEGContainer X_test%d: ' %i) + str(self.X_test[str(i)].shape))
      print(('EEGContainer y_test%d: ' %i) + str(self.y_test[str(i)].shape))

###Convolutional Neural Network

###CNN

In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d( 1, 16, (1, 11), stride=(1, 1), padding=0)
    self.conv2 = nn.Conv2d(16, 16, (22, 1), stride=(1, 1), padding=0)
    self.conv3 = nn.Conv2d( 1, 16, (1, 11), stride=(1, 1), padding=0)
    self.conv4 = nn.Conv2d(16, 16, (16, 1), stride=(1, 1), padding=0)
    self.conv5 = nn.Conv2d( 1, 16, 3, stride=1, padding=1)
    self.conv6 = nn.Conv2d(16, 32, 3, stride=1, padding=1)
    self.conv7 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
    self.fc1 = nn.Linear(64 * 2 * 10, 200)
    self.fc2 = nn.Linear(200, 100)
    self.fc3 = nn.Linear(100, 4)
  
  def forward(self, x):
    dropval = 0.7
    x = F.dropout2d(F.relu(self.conv1(x)), p=0.2)
    x = F.dropout2d(F.relu(self.conv2(x)), p=dropval)
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 3), (1, 3))
    x = F.dropout2d(F.relu(self.conv3(x)), p=dropval)
    x = F.dropout2d(F.relu(self.conv4(x)), p=dropval)
    x = x.permute(0, 2, 1, 3)
    x = F.max_pool2d(x, (1, 4), (1, 4))
    x = F.dropout2d(F.relu(self.conv5(x)), p=dropval)
    x = F.max_pool2d(x, 2, 2)
    x = F.dropout2d(F.relu(self.conv6(x)), p=dropval)
    x = F.max_pool2d(x, 2, 2)
    x = F.dropout2d(F.relu(self.conv7(x)), p=dropval)
    x = F.max_pool2d(x, 2, 2)
    x = x.view(-1, 64 * 2 * 10)
    x = F.dropout(F.relu(self.fc1(x)), p=dropval)
    x = F.dropout(F.relu(self.fc2(x)), p=dropval)
    x = self.fc3(x)
    return x

In [92]:
'''
# N x 1 x 22 x 534
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d( 1, 40, (1, 25))
    #nn.init.xavier_uniform(self.conv1.weight)
    self.bnc1 = nn.BatchNorm2d(40)
    self.conv2 = nn.Conv2d(40, 40, (22, 1))
    #nn.init.xavier_uniform(self.conv2.weight)
    self.bnc2 = nn.BatchNorm2d(40)
    self.fc = nn.Linear(30 * 40, 4)
    #nn.init.xavier_uniform(self.fc.weight)
  
  def forward(self, x):
    x = F.elu(self.bnc1(self.conv1(x)))
    x = F.elu(self.bnc2(self.conv2(x)))
    x = F.dropout2d(x, p=0.5)
    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 75), stride=(1, 15))
    x = x.view(-1, 30 * 40)
    x = self.fc(x)
    return x
'''

'\n# N x 1 x 22 x 534\nclass CNN(nn.Module):\n  def __init__(self):\n    super(CNN, self).__init__()\n    self.conv1 = nn.Conv2d( 1, 40, (1, 25))\n    #nn.init.xavier_uniform(self.conv1.weight)\n    self.bnc1 = nn.BatchNorm2d(40)\n    self.conv2 = nn.Conv2d(40, 40, (22, 1))\n    #nn.init.xavier_uniform(self.conv2.weight)\n    self.bnc2 = nn.BatchNorm2d(40)\n    self.fc = nn.Linear(30 * 40, 4)\n    #nn.init.xavier_uniform(self.fc.weight)\n  \n  def forward(self, x):\n    x = F.elu(self.bnc1(self.conv1(x)))\n    x = F.elu(self.bnc2(self.conv2(x)))\n    x = F.dropout2d(x, p=0.5)\n    x = F.max_pool2d(x.permute(0, 2, 1, 3), (1, 75), stride=(1, 15))\n    x = x.view(-1, 30 * 40)\n    x = self.fc(x)\n    return x\n'

##Setup

In [0]:
data_dir = 'drive/ee239as/project_datasets'
batch_size = 36
train_subject = 9
time_batch = 534

In [164]:
EEGset = EEGMinimalContainer(data_dir, train_subject)

EEGContainer X_train: (278, 1, 22, 1000)
EEGContainer y_train: (278,)
EEGContainer X_test1: (287, 1, 22, 1000)
EEGContainer y_test1: (287,)
EEGContainer X_test2: (286, 1, 22, 1000)
EEGContainer y_test2: (286,)
EEGContainer X_test3: (286, 1, 22, 1000)
EEGContainer y_test3: (286,)
EEGContainer X_test4: (284, 1, 22, 1000)
EEGContainer y_test4: (284,)
EEGContainer X_test5: (282, 1, 22, 1000)
EEGContainer y_test5: (282,)
EEGContainer X_test6: (285, 1, 22, 1000)
EEGContainer y_test6: (285,)
EEGContainer X_test7: (288, 1, 22, 1000)
EEGContainer y_test7: (288,)
EEGContainer X_test8: (282, 1, 22, 1000)
EEGContainer y_test8: (282,)


In [0]:
train_loader = torch.utils.data.DataLoader(EEGset.train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = {}
for i in range(1, 10):
  if train_subject == i:
    continue
  test_loader[str(i)] = torch.utils.data.DataLoader(EEGset.test_dataset[str(i)],
                                                    batch_size=batch_size,
                                                    shuffle=False)

In [0]:
num_epochs = 200
learning_rate = 1e-4

use_cuda = True

net = CNN()
criterion = nn.CrossEntropyLoss()

In [0]:
if use_cuda and torch.cuda.is_available():
  net.cuda()

optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=1e-3)

##Training

###For 1000 time units

In [168]:
training_acc_arr = np.empty(num_epochs)
testing_acc_arr = np.empty((9, num_epochs))

for epoch in range(num_epochs):
  
  net.train()
  
  for i, (signals, labels) in enumerate(train_loader):
    
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals)
    labels = labels.type(torch.LongTensor)
    labels = Variable(torch.squeeze(labels))
    
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels = labels.cuda()
    
    optimizer.zero_grad()
    outputs = net(signals)
    
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 2 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
            % (epoch+1, num_epochs, i+1, len(EEGset.train_dataset)//batch_size, 
               loss.data[0]))
  
  net.eval()
  
  # Training accuracy
  total = 0
  correct = 0
  for signals, labels in train_loader:
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals)
    labels = torch.squeeze(labels.type(torch.LongTensor))
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels = labels.cuda()
    outputs = net(signals)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
  training_acc_arr[epoch] = (correct/total)
  print ('Training Accuracy: %.5f' % training_acc_arr[epoch])
  
  
  # Testing accuracy
  print ('Testing Accuracy: [', end='')
  avg_total = 0
  avg_correct = 0
  for subject in range(9):
    if subject + 1 == train_subject:
      print ('x ', end='')
      continue
    total = 0
    correct = 0
    for signals, labels in test_loader[str(subject+1)]:
      signals = signals.type(torch.FloatTensor)
      signals = Variable(signals)
      labels = torch.squeeze(labels.type(torch.LongTensor))
      if use_cuda and torch.cuda.is_available():
        signals = signals.cuda()
        labels = labels.cuda()
      outputs = net(signals)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum()
    testing_acc_arr[subject, epoch] = (correct/total)
    print ('%.5f ' %(correct/total), end='')
    avg_total += total
    avg_correct += correct
  print (']')
  print ('Testing Accuracy Average: %.5f' % (avg_correct/avg_total))

Epoch [1/200], Step [2/7], Loss: 1.3777
Epoch [1/200], Step [4/7], Loss: 1.4018
Epoch [1/200], Step [6/7], Loss: 1.3832
Epoch [1/200], Step [8/7], Loss: 1.3940
Training Accuracy: 0.24460
Testing Accuracy: [0.25087 0.24476 0.24825 0.25000 0.25532 0.24912 0.25000 0.25532 x ]
Testing Accuracy Average: 0.25044
Epoch [2/200], Step [2/7], Loss: 1.3808
Epoch [2/200], Step [4/7], Loss: 1.3688
Epoch [2/200], Step [6/7], Loss: 1.3854
Epoch [2/200], Step [8/7], Loss: 1.3946
Training Accuracy: 0.24460
Testing Accuracy: [0.25087 0.24476 0.24825 0.25000 0.25532 0.24912 0.25000 0.25532 x ]
Testing Accuracy Average: 0.25044
Epoch [3/200], Step [2/7], Loss: 1.3973
Epoch [3/200], Step [4/7], Loss: 1.3876
Epoch [3/200], Step [6/7], Loss: 1.3743
Epoch [3/200], Step [8/7], Loss: 1.3740
Training Accuracy: 0.24460
Testing Accuracy: [0.25087 0.24476 0.24825 0.25000 0.25532 0.24912 0.25000 0.25532 x ]
Testing Accuracy Average: 0.25044
Epoch [4/200], Step [2/7], Loss: 1.3859
Epoch [4/200], Step [4/7], Loss: 1.3

0.26736 0.32624 x ]
Testing Accuracy Average: 0.30614
Epoch [13/200], Step [2/7], Loss: 1.2366
Epoch [13/200], Step [4/7], Loss: 1.2221
Epoch [13/200], Step [6/7], Loss: 1.3253
Epoch [13/200], Step [8/7], Loss: 1.3210
Training Accuracy: 0.50719
Testing Accuracy: [0.33101 0.31119 0.31818 0.30986 0.28014 0.26667 0.28819 0.34397 x ]
Testing Accuracy Average: 0.30614
Epoch [14/200], Step [2/7], Loss: 1.3193
Epoch [14/200], Step [4/7], Loss: 1.0889
Epoch [14/200], Step [6/7], Loss: 1.1387
Epoch [14/200], Step [8/7], Loss: 1.0991
Training Accuracy: 0.46763
Testing Accuracy: [0.31010 0.29720 0.30070 0.35915 0.28369 0.24561 0.27778 0.32624 x ]
Testing Accuracy Average: 0.30000
Epoch [15/200], Step [2/7], Loss: 1.1277
Epoch [15/200], Step [4/7], Loss: 1.1799
Epoch [15/200], Step [6/7], Loss: 1.3172
Epoch [15/200], Step [8/7], Loss: 1.2206
Training Accuracy: 0.44604
Testing Accuracy: [0.34146 0.33916 0.29371 0.36268 0.29787 0.24211 0.27778 0.31915 x ]
Testing Accuracy Average: 0.30921
Epoch [16/

0.34859 0.26241 0.22105 0.24653 0.32270 x ]
Testing Accuracy Average: 0.29254
Epoch [25/200], Step [2/7], Loss: 0.8424
Epoch [25/200], Step [4/7], Loss: 0.9901
Epoch [25/200], Step [6/7], Loss: 0.9131
Epoch [25/200], Step [8/7], Loss: 0.9988
Training Accuracy: 0.64029
Testing Accuracy: [0.29268 0.22378 0.30769 0.30282 0.25887 0.28070 0.21875 0.30851 x ]
Testing Accuracy Average: 0.27412
Epoch [26/200], Step [2/7], Loss: 0.8275
Epoch [26/200], Step [4/7], Loss: 0.8964
Epoch [26/200], Step [6/7], Loss: 1.1592
Epoch [26/200], Step [8/7], Loss: 0.9194
Training Accuracy: 0.65108
Testing Accuracy: [0.33101 0.25524 0.34266 0.35563 0.25532 0.24561 0.22569 0.32624 x ]
Testing Accuracy Average: 0.29211
Epoch [27/200], Step [2/7], Loss: 0.9181
Epoch [27/200], Step [4/7], Loss: 1.0671
Epoch [27/200], Step [6/7], Loss: 0.9291
Epoch [27/200], Step [8/7], Loss: 0.8475
Training Accuracy: 0.65468
Testing Accuracy: [0.32404 0.21678 0.33566 0.31690 0.26596 0.23158 0.22917 0.32270 x ]
Testing Accuracy Ave

0.28169 0.25887 0.30175 0.21528 0.35461 x ]
Testing Accuracy Average: 0.29079
Epoch [37/200], Step [2/7], Loss: 0.6881
Epoch [37/200], Step [4/7], Loss: 0.8126
Epoch [37/200], Step [6/7], Loss: 0.7892
Epoch [37/200], Step [8/7], Loss: 0.6732
Training Accuracy: 0.71583
Testing Accuracy: [0.34495 0.24825 0.33566 0.28521 0.25532 0.24561 0.21875 0.34043 x ]
Testing Accuracy Average: 0.28421
Epoch [38/200], Step [2/7], Loss: 0.7314
Epoch [38/200], Step [4/7], Loss: 0.8533
Epoch [38/200], Step [6/7], Loss: 0.7679
Epoch [38/200], Step [8/7], Loss: 0.5792
Training Accuracy: 0.75180
Testing Accuracy: [0.33101 0.25874 0.34266 0.27465 0.25532 0.28772 0.21528 0.35106 x ]
Testing Accuracy Average: 0.28947
Epoch [39/200], Step [2/7], Loss: 0.5561
Epoch [39/200], Step [4/7], Loss: 0.7199
Epoch [39/200], Step [6/7], Loss: 0.7042
Epoch [39/200], Step [8/7], Loss: 0.5737
Training Accuracy: 0.75899
Testing Accuracy: [0.31010 0.26224 0.30420 0.27465 0.25532 0.29825 0.22222 0.36525 x ]
Testing Accuracy Ave

0.28169 0.25887 0.25263 0.22569 0.36525 x ]
Testing Accuracy Average: 0.29342
Epoch [49/200], Step [2/7], Loss: 0.5263
Epoch [49/200], Step [4/7], Loss: 0.6195
Epoch [49/200], Step [6/7], Loss: 0.5159
Epoch [49/200], Step [8/7], Loss: 0.6647
Training Accuracy: 0.81655
Testing Accuracy: [0.35192 0.25874 0.32517 0.27817 0.25887 0.28421 0.23264 0.36879 x ]
Testing Accuracy Average: 0.29474
Epoch [50/200], Step [2/7], Loss: 0.6539
Epoch [50/200], Step [4/7], Loss: 0.5057
Epoch [50/200], Step [6/7], Loss: 0.7032
Epoch [50/200], Step [8/7], Loss: 0.4115
Training Accuracy: 0.79137
Testing Accuracy: [0.31707 0.24825 0.32867 0.26761 0.26241 0.27368 0.22917 0.33688 x ]
Testing Accuracy Average: 0.28289
Epoch [51/200], Step [2/7], Loss: 0.3709
Epoch [51/200], Step [4/7], Loss: 0.5415
Epoch [51/200], Step [6/7], Loss: 0.4099
Epoch [51/200], Step [8/7], Loss: 0.6357
Training Accuracy: 0.83094
Testing Accuracy: [0.30314 0.25874 0.29371 0.25000 0.24823 0.30175 0.22569 0.32979 x ]
Testing Accuracy Ave

...

0.36585 0.25175 0.32517 0.26408 0.25177 0.25614 0.22569 0.39362 x ]
Testing Accuracy Average: 0.29167
Epoch [145/200], Step [2/7], Loss: 0.0076
Epoch [145/200], Step [4/7], Loss: 0.0113
Epoch [145/200], Step [6/7], Loss: 0.0056
Epoch [145/200], Step [8/7], Loss: 0.0065
Training Accuracy: 1.00000
Testing Accuracy: [0.36934 0.24126 0.31818 0.26056 0.25177 0.25263 0.22917 0.37943 x ]
Testing Accuracy Average: 0.28772
Epoch [146/200], Step [2/7], Loss: 0.0079
Epoch [146/200], Step [4/7], Loss: 0.0116
Epoch [146/200], Step [6/7], Loss: 0.0057
Epoch [146/200], Step [8/7], Loss: 0.0068
Training Accuracy: 1.00000
Testing Accuracy: [0.36585 0.24825 0.32517 0.26408 0.24823 0.24912 0.22222 0.39007 x ]
Testing Accuracy Average: 0.28904
Epoch [147/200], Step [2/7], Loss: 0.0072
Epoch [147/200], Step [4/7], Loss: 0.0050
Epoch [147/200], Step [6/7], Loss: 0.0070
Epoch [147/200], Step [8/7], Loss: 0.0041
Training Accuracy: 1.00000
Testing Accuracy: [0.36934 0.24126 0.32517 0.26408 0.24468 0.25263 0.22

0.36237 0.25175 0.32517 0.26761 0.24823 0.25965 0.23264 0.38652 x ]
Testing Accuracy Average: 0.29167
Epoch [157/200], Step [2/7], Loss: 0.0080
Epoch [157/200], Step [4/7], Loss: 0.0078
Epoch [157/200], Step [6/7], Loss: 0.0047
Epoch [157/200], Step [8/7], Loss: 0.0056
Training Accuracy: 1.00000
Testing Accuracy: [0.36585 0.25175 0.32168 0.26408 0.25177 0.25263 0.22917 0.39007 x ]
Testing Accuracy Average: 0.29079
Epoch [158/200], Step [2/7], Loss: 0.0063
Epoch [158/200], Step [4/7], Loss: 0.0048
Epoch [158/200], Step [6/7], Loss: 0.0059
Epoch [158/200], Step [8/7], Loss: 0.0058
Training Accuracy: 1.00000
Testing Accuracy: [0.36585 0.24825 0.32517 0.26408 0.24468 0.25263 0.22917 0.38298 x ]
Testing Accuracy Average: 0.28904
Epoch [159/200], Step [2/7], Loss: 0.0046
Epoch [159/200], Step [4/7], Loss: 0.0036
Epoch [159/200], Step [6/7], Loss: 0.0098
Epoch [159/200], Step [8/7], Loss: 0.0078
Training Accuracy: 1.00000
Testing Accuracy: [0.36585 0.25175 0.32168 0.26761 0.25177 0.24912 0.22

0.36585 0.23427 0.33916 0.26408 0.25177 0.25263 0.22917 0.39362 x ]
Testing Accuracy Average: 0.29123
Epoch [169/200], Step [2/7], Loss: 0.0041
Epoch [169/200], Step [4/7], Loss: 0.0042
Epoch [169/200], Step [6/7], Loss: 0.0034
Epoch [169/200], Step [8/7], Loss: 0.0060
Training Accuracy: 1.00000
Testing Accuracy: [0.36934 0.26224 0.32867 0.26761 0.24823 0.25263 0.22917 0.38298 x ]
Testing Accuracy Average: 0.29254
Epoch [170/200], Step [2/7], Loss: 0.0055
Epoch [170/200], Step [4/7], Loss: 0.0039
Epoch [170/200], Step [6/7], Loss: 0.0037
Epoch [170/200], Step [8/7], Loss: 0.0033
Training Accuracy: 1.00000
Testing Accuracy: [0.36585 0.24126 0.33217 0.26408 0.25177 0.25263 0.22222 0.40071 x ]
Testing Accuracy Average: 0.29123
Epoch [171/200], Step [2/7], Loss: 0.0036
Epoch [171/200], Step [4/7], Loss: 0.0049
Epoch [171/200], Step [6/7], Loss: 0.0027
Epoch [171/200], Step [8/7], Loss: 0.0044
Training Accuracy: 1.00000
Testing Accuracy: [0.36585 0.25175 0.31818 0.26408 0.24823 0.25263 0.23

0.36237 0.25874 0.31818 0.26408 0.25177 0.24912 0.22569 0.38652 x ]
Testing Accuracy Average: 0.28947
Epoch [181/200], Step [2/7], Loss: 0.0027
Epoch [181/200], Step [4/7], Loss: 0.0043
Epoch [181/200], Step [6/7], Loss: 0.0028
Epoch [181/200], Step [8/7], Loss: 0.0040
Training Accuracy: 1.00000
Testing Accuracy: [0.35540 0.23776 0.32867 0.27113 0.25177 0.25614 0.23264 0.38652 x ]
Testing Accuracy Average: 0.28991
Epoch [182/200], Step [2/7], Loss: 0.0029
Epoch [182/200], Step [4/7], Loss: 0.0037
Epoch [182/200], Step [6/7], Loss: 0.0019
Epoch [182/200], Step [8/7], Loss: 0.0042
Training Accuracy: 1.00000
Testing Accuracy: [0.36934 0.25524 0.32517 0.26408 0.24468 0.25263 0.23264 0.38652 x ]
Testing Accuracy Average: 0.29123
Epoch [183/200], Step [2/7], Loss: 0.0049
Epoch [183/200], Step [4/7], Loss: 0.0029
Epoch [183/200], Step [6/7], Loss: 0.0019
Epoch [183/200], Step [8/7], Loss: 0.0034
Training Accuracy: 1.00000
Testing Accuracy: [0.35540 0.23776 0.33217 0.27113 0.25177 0.25614 0.23

0.36585 0.24825 0.32517 0.26056 0.25177 0.24912 0.22917 0.39007 x ]
Testing Accuracy Average: 0.28991
Epoch [193/200], Step [2/7], Loss: 0.0028
Epoch [193/200], Step [4/7], Loss: 0.0029
Epoch [193/200], Step [6/7], Loss: 0.0020
Epoch [193/200], Step [8/7], Loss: 0.0018
Training Accuracy: 1.00000
Testing Accuracy: [0.35889 0.24825 0.32867 0.26408 0.25177 0.25263 0.23264 0.38652 x ]
Testing Accuracy Average: 0.29035
Epoch [194/200], Step [2/7], Loss: 0.0024
Epoch [194/200], Step [4/7], Loss: 0.0024
Epoch [194/200], Step [6/7], Loss: 0.0032
Epoch [194/200], Step [8/7], Loss: 0.0019
Training Accuracy: 1.00000
Testing Accuracy: [0.36585 0.25175 0.32517 0.26408 0.25177 0.24912 0.22917 0.39007 x ]
Testing Accuracy Average: 0.29079
Epoch [195/200], Step [2/7], Loss: 0.0034
Epoch [195/200], Step [4/7], Loss: 0.0024
Epoch [195/200], Step [6/7], Loss: 0.0024
Epoch [195/200], Step [8/7], Loss: 0.0024
Training Accuracy: 1.00000
Testing Accuracy: [0.35889 0.24126 0.32517 0.26408 0.25177 0.25263 0.22

###For time_batch units

In [169]:
'''
training_acc_arr = np.zeros(num_epochs)
testing_acc_arr = np.zeros((9, num_epochs))

for epoch in range(num_epochs):
  
  net.train()
  
  total = 0
  correct = 0
  
  for i, (signals, labels) in enumerate(train_loader):
    
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals)
    labels = labels.type(torch.LongTensor)
    labels_check = torch.squeeze(labels)
    labels = Variable(torch.squeeze(labels))
    loc = np.random.choice(1000-time_batch, 4, replace=False)
    s0 = signals[:, :, :, loc[0]:(loc[0]+time_batch)]
    s1 = signals[:, :, :, loc[1]:(loc[1]+time_batch)]
    s2 = signals[:, :, :, loc[2]:(loc[2]+time_batch)]
    s3 = signals[:, :, :, loc[3]:(loc[3]+time_batch)]
    signals = torch.cat((s0, s1, s2, s3), 0)
    labels_more = torch.cat([labels] * 4)
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels_more = labels_more.cuda()
    optimizer.zero_grad()
    outputs = net(signals)
    loss = criterion(outputs, labels_more)
    loss.backward()
    optimizer.step()
    _, predicted = torch.max(outputs.data, 1)
    predicted = predicted.cpu()
    predicted = predicted.view(-1, 4).numpy()
    predicted, _ = stats.mode(predicted, axis=1)
    predicted = np.squeeze(predicted)
    total += labels.size(0)
    correct += (predicted == labels_check.numpy()).sum()
    
    if (i+1) % 8 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
            % (epoch+1, num_epochs, i+1, len(EEGset.train_dataset)//batch_size, 
               loss.data[0]))
  
  net.eval()
  
  # Training accuracy
  total = 0
  correct = 0
  for signals, labels in train_loader:
    signals = signals.type(torch.FloatTensor)
    signals = Variable(signals)
    labels = labels.type(torch.LongTensor)
    labels_check = torch.squeeze(labels)
    labels = Variable(torch.squeeze(labels))
    loc = np.random.choice(1000-time_batch, 4, replace=False)
    s0 = signals[:, :, :, loc[0]:(loc[0]+time_batch)]
    s1 = signals[:, :, :, loc[1]:(loc[1]+time_batch)]
    s2 = signals[:, :, :, loc[2]:(loc[2]+time_batch)]
    s3 = signals[:, :, :, loc[3]:(loc[3]+time_batch)]
    signals = torch.cat((s0, s1, s2, s3), 0)
    labels_more = torch.cat([labels] * 4)
    if use_cuda and torch.cuda.is_available():
      signals = signals.cuda()
      labels_more = labels_more.cuda()
    outputs = net(signals)
    _, predicted = torch.max(outputs.data, 1)
    predicted = predicted.cpu()
    predicted = predicted.view(-1, 4).numpy()
    predicted, _ = stats.mode(predicted, axis=1)
    predicted = np.squeeze(predicted)
    total += labels.size(0)
    correct += (predicted == labels_check.numpy()).sum()
  training_acc_arr[epoch] = (correct/total)
  print ('Training Accuracy: %.5f' % training_acc_arr[epoch])
  
  # Testing accuracy
  print ('Testing Accuracy: [', end='')
  for subject in range(9):
    if subject + 1 == train_subject:
      print ('x ', end='')
      continue
    total = 0
    correct = 0
    for signals, labels in test_loader[str(subject+1)]:
      signals = signals.type(torch.FloatTensor)
      signals = Variable(signals)
      labels = labels.type(torch.LongTensor)
      labels_check = torch.squeeze(labels)
      labels = Variable(torch.squeeze(labels))
      loc = np.random.choice(1000-time_batch, 4, replace=False)
      s0 = signals[:, :, :, loc[0]:(loc[0]+time_batch)]
      s1 = signals[:, :, :, loc[1]:(loc[1]+time_batch)]
      s2 = signals[:, :, :, loc[2]:(loc[2]+time_batch)]
      s3 = signals[:, :, :, loc[3]:(loc[3]+time_batch)]
      signals = torch.cat((s0, s1, s2, s3), 0)
      labels_more = torch.cat([labels] * 4)
      if use_cuda and torch.cuda.is_available():
        signals = signals.cuda()
        labels_more = labels_more.cuda()
      outputs = net(signals)
      _, predicted = torch.max(outputs.data, 1)
      predicted = predicted.cpu()
      predicted = predicted.view(-1, 4).numpy()
      predicted, _ = stats.mode(predicted, axis=1)
      predicted = np.squeeze(predicted)
      total += labels.size(0)
      correct += (predicted == labels_check.numpy()).sum()
    testing_acc_arr[subject, epoch] = (correct/total)
    print ('%.5f ' %(correct/total), end='')
  print (']')
  print ('Testing Accuracy Average: %.5f' % np.average(testing_acc_arr[np.delete(np.arange(9), train_subject - 1), epoch]))
'''

"\ntraining_acc_arr = np.zeros(num_epochs)\ntesting_acc_arr = np.zeros((9, num_epochs))\n\nfor epoch in range(num_epochs):\n  \n  net.train()\n  \n  total = 0\n  correct = 0\n  \n  for i, (signals, labels) in enumerate(train_loader):\n    \n    signals = signals.type(torch.FloatTensor)\n    signals = Variable(signals)\n    labels = labels.type(torch.LongTensor)\n    labels_check = torch.squeeze(labels)\n    labels = Variable(torch.squeeze(labels))\n    loc = np.random.choice(1000-time_batch, 4, replace=False)\n    s0 = signals[:, :, :, loc[0]:(loc[0]+time_batch)]\n    s1 = signals[:, :, :, loc[1]:(loc[1]+time_batch)]\n    s2 = signals[:, :, :, loc[2]:(loc[2]+time_batch)]\n    s3 = signals[:, :, :, loc[3]:(loc[3]+time_batch)]\n    signals = torch.cat((s0, s1, s2, s3), 0)\n    labels_more = torch.cat([labels] * 4)\n    if use_cuda and torch.cuda.is_available():\n      signals = signals.cuda()\n      labels_more = labels_more.cuda()\n    optimizer.zero_grad()\n    outputs = net(signals)\n

###Print MATLAB Format

In [170]:
# Print Training Accuracy
print('Training_Accuracy = [', end='')
for acc in np.nditer(training_acc_arr):
  print('%.5f, ' % acc, end='')
print('];')

# Print Testing Accuracy
print('Testing_Accuracy = [', end='')
for subject in range(9):
  for acc in np.nditer(testing_acc_arr[subject, :]):
    print('%.5f, ' % acc, end='')
  print('; ', end='')
print('];')

Training_Accuracy = [0.24460, 0.24460, 0.24460, 0.24460, 0.24460, 0.24820, 0.24460, 0.28417, 0.32734, 0.36691, 0.39928, 0.43525, 0.50719, 0.46763, 0.44604, 0.51079, 0.51079, 0.53957, 0.57914, 0.55036, 0.58273, 0.58633, 0.61871, 0.61871, 0.64029, 0.65108, 0.65468, 0.61151, 0.64748, 0.66187, 0.71223, 0.72302, 0.71223, 0.70504, 0.73381, 0.74101, 0.71583, 0.75180, 0.75899, 0.72662, 0.72662, 0.76978, 0.79496, 0.78417, 0.79856, 0.77698, 0.80216, 0.78058, 0.81655, 0.79137, 0.83094, 0.82374, 0.84532, 0.85252, 0.84173, 0.80935, 0.83453, 0.85252, 0.84532, 0.87050, 0.87770, 0.86331, 0.87410, 0.87770, 0.88129, 0.89209, 0.89928, 0.91007, 0.92446, 0.92806, 0.91727, 0.93165, 0.88489, 0.94604, 0.94604, 0.96043, 0.94604, 0.94604, 0.91007, 0.88489, 0.96043, 0.97842, 0.97482, 0.97482, 0.97122, 0.97482, 0.98201, 0.91007, 0.94604, 0.98201, 0.99281, 0.98921, 0.99640, 0.98561, 0.99640, 0.99281, 0.99281, 0.99281, 0.99640, 0.99640, 0.99281, 0.99640, 0.98921, 0.99640, 0.99281, 0.99281, 0.99640, 0.99640, 1.00000